In [1]:
import shutil
import os

import pandas as pd
from sklearn.utils import shuffle
import matplotlib.patches as patches
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf


import keras
import keras.backend as K
from keras.models import Model
from keras import backend as K
from keras.utils import np_utils
from keras.optimizers import SGD, Adam
from keras.datasets import cifar10
from keras.layers.core import Layer
from keras.models import model_from_json
from keras.callbacks import LearningRateScheduler
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Input, concatenate, GlobalAveragePooling2D, AveragePooling2D,Flatten

import numpy as np 
import math 

Using TensorFlow backend.


In [14]:
#save test images in a separate folder (test_file)

def save_test():
    df_2 = pd.read_csv('test.csv')
    testfileslist = df_2['image_name'].tolist()
    testfileslist = ["F:\Flipkart Grid\Round 2\images/" + s for s in testfileslist]
    dest = "test_file/"
    print(testfileslist[2990])
    for i,file_name in enumerate(testfileslist):
        shutil.copy(file_name, dest)
        if i %250 == 0: print(i)
            
save_test()

F:\Flipkart Grid\Round 2\images/147437560540111469528221831-Roadster-Navy--Maroon-Striped-Sweater-5571469528221194-1.png
0
250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
3000
3250
3500
3750
4000
4250
4500
4750
5000
5250
5500
5750
6000
6250
6500
6750
7000
7250
7500
7750
8000
8250
8500
8750
9000
9250
9500
9750
10000
10250
10500
10750
11000
11250
11500
11750
12000
12250
12500
12750
13000
13250
13500
13750
14000
14250
14500
14750
15000
15250
15500
15750
16000
16250
16500
16750
17000
17250
17500
17750
18000
18250
18500
18750
19000
19250
19500
19750
20000
20250
20500
20750
21000
21250
21500
21750
22000
22250
22500
22750
23000
23250
23500
23750
24000


In [2]:
# Load the trained model and predict on test data

initial_lrate = 0.025
sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)
json_file = open('F:\Flipkart Grid\Round 3/Models/model_v2.json', 'r')
model = json_file.read()
json_file.close()
model = model_from_json(model)
model.load_weights('F:\Flipkart Grid\Round 3/Models/model_v2.h5')
model.compile(loss='mean_absolute_error',  optimizer=sgd)

In [3]:
#Function to load test data in batches

def load_test_data(index, minibatchsize, testfileslist, last=False):
    images = []
    if last == False:
          batchlist = testfileslist[index*minibatchsize:(index+1)*minibatchsize]
    else:
          batchlist = testfileslist[index*minibatchsize:]
    for i,f in enumerate(batchlist):
        image = Image.open(f)
        image = image.resize((224, 224))
        image = np.array(image)
        images.append(image)
    images = np.array(images)

    X_train = images.astype('float32')
    X_train = (X_train) / 255.0

    return X_train

# Function to get predictions from Test Data 
# Note=> (Please modify the path to the required one while predicting from another machine)
def eval_test():
    minibatchsize = 10
    df_2 = pd.read_csv('test.csv')
    testfileslist = df_2['image_name'].tolist()
    testfileslist = ["F:\Flipkart Grid\Round 2\images/" + s for s in testfileslist]
    print(len(testfileslist))
    for j in range(1):
        for i in range(int(len(testfileslist)/minibatchsize)):
            X_test = load_test_data(i, minibatchsize, testfileslist, last = False)
            predict = model.predict(X_test, batch_size=None, verbose=0, steps=None)            
            if i ==0 : predcoords = predict
            else: predcoords = np.concatenate((predcoords,predict),axis=1)
            if i%10 == 0: print(np.shape(predcoords))
    X_test = load_test_data(i, minibatchsize, testfileslist, last = True)
    predict = model.predict(X_test, batch_size=None, verbose=0, steps=None)
    predcoords = np.concatenate((predcoords,predict),axis=1)
    
    return predcoords

testpredcoords = eval_test()

24045
(2, 10, 2)
(2, 110, 2)
(2, 210, 2)
(2, 310, 2)
(2, 410, 2)
(2, 510, 2)
(2, 610, 2)
(2, 710, 2)
(2, 810, 2)
(2, 910, 2)
(2, 1010, 2)
(2, 1110, 2)
(2, 1210, 2)
(2, 1310, 2)
(2, 1410, 2)
(2, 1510, 2)
(2, 1610, 2)
(2, 1710, 2)
(2, 1810, 2)
(2, 1910, 2)
(2, 2010, 2)
(2, 2110, 2)
(2, 2210, 2)
(2, 2310, 2)
(2, 2410, 2)
(2, 2510, 2)
(2, 2610, 2)
(2, 2710, 2)
(2, 2810, 2)
(2, 2910, 2)
(2, 3010, 2)
(2, 3110, 2)
(2, 3210, 2)
(2, 3310, 2)
(2, 3410, 2)
(2, 3510, 2)
(2, 3610, 2)
(2, 3710, 2)
(2, 3810, 2)
(2, 3910, 2)
(2, 4010, 2)
(2, 4110, 2)
(2, 4210, 2)
(2, 4310, 2)
(2, 4410, 2)
(2, 4510, 2)
(2, 4610, 2)
(2, 4710, 2)
(2, 4810, 2)
(2, 4910, 2)
(2, 5010, 2)
(2, 5110, 2)
(2, 5210, 2)
(2, 5310, 2)
(2, 5410, 2)
(2, 5510, 2)
(2, 5610, 2)
(2, 5710, 2)
(2, 5810, 2)
(2, 5910, 2)
(2, 6010, 2)
(2, 6110, 2)
(2, 6210, 2)
(2, 6310, 2)
(2, 6410, 2)
(2, 6510, 2)
(2, 6610, 2)
(2, 6710, 2)
(2, 6810, 2)
(2, 6910, 2)
(2, 7010, 2)
(2, 7110, 2)
(2, 7210, 2)
(2, 7310, 2)
(2, 7410, 2)
(2, 7510, 2)
(2, 7610, 2)
(2, 

In [4]:
#Convert Predictions to the required format and save as CSV file for submission

def testcsv():
    x1 = (testpredcoords[0,:,0]-(testpredcoords[1,:,0]/2))*640
    y1 = (testpredcoords[0,:,1]-(testpredcoords[1,:,1]/2))*480
    x2 = (testpredcoords[0,:,0]+(testpredcoords[1,:,0]/2))*640
    y2 = (testpredcoords[0,:,1]+(testpredcoords[1,:,1]/2))*480
    df_2 = pd.read_csv('test.csv')
    testcsv = pd.DataFrame({'image_name':df_2['image_name'], 'x1':x1[:24045], 'y1':y1[:24045], 'x2':x2[:24045], 'y2':y2[:24045]})
    testcsv.to_csv('23219_v1.csv', index=False)
    
testcsv()

In [ ]:
# Plot the predictions on Test Data to check the Model

def plot_image(i):
    df_2 = pd.read_csv('test.csv')
    testfileslist = df_2['image_name'].tolist()
    testfileslist = ["F:\Flipkart Grid\Round 2\images/" + s for s in testfileslist]
    img = cv2.imread(testfileslist[i])
    fig,ax = plt.subplots(1)
    ax.imshow(img)
    x = (testpredcoords[0][i][0]-(testpredcoords[1][i][0]/2))*640
    y = (testpredcoords[0][i][1]-(testpredcoords[1][i][1]/2))*480
    w = (testpredcoords[1][i][0])*640
    h = (testpredcoords[1][i][1])*480
    rect = patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='g',facecolor='none')
    ax.add_patch(rect)
    plt.show()
    
plot_image(10)
plot_image(20)
plot_image(30)
plot_image(40)